# 使用 LCEL 建立簡單的 LLM 申請

在本快速入門中將向您展示如何使用 LangChain 建立簡單的 LLM 應用程式。該應用程式會將英語文字翻譯成另一種語言。這是一個相對簡單的 LLM 應用 - 它只是一個 LLM 呼叫加上一些提示。儘管如此，這仍然是開始使用 LangChain 的好方法 - 只需一些提示和 LLM 呼叫就可以建立很多功能！

閱讀本教學後，您將對以下內容有一個整體的了解：

使用[大語言模型](https://python.langchain.com/v0.2/docs/concepts/#chat-models)

使用[PromptTemplates](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates)和[OutputParsers](https://python.langchain.com/v0.2/docs/concepts/#output-parsers)

使用[LangChain 表達式語言 (LCEL)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel)將元件連結在一起

使用[LangSmith](https://python.langchain.com/v0.2/docs/concepts/#langsmith)調試和追蹤您的應用程式

使用[LangServe](https://python.langchain.com/v0.2/docs/concepts/#langserve)部署您的應用程式

讓我們深入了解吧！

In [2]:
!pip install langchain
!pip install langchain-huggingface
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-man

有關更多詳細信息，請參閱我們的[安裝指南](https://python.langchain.com/v0.2/docs/how_to/installation/)。

您使用 LangChain 建立的許多應用程式將包含多個步驟，並多次呼叫 LLM 調用。隨著這些應用程式變得越來越複雜，能夠檢查鍊或代理內部到底發生了什麼變得至關重要。最好的方法是使用[LangSmith](https://smith.langchain.com/)。

在上面的連結註冊後，請確保設定環境變數以開始記錄追蹤：

In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [27]:

LANGCHAIN_API_KEY="替換為你的LANGCHAIN_API_KEY"

In [5]:
import getpass
import os
from huggingface_hub import login, HfApi
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

# 替換為你的Hugging Face API Token
my_read_token = "替換為你的Hugging Face API Token"
os.environ["HUGGINGFACE_TOKEN"] = my_read_token
# 登錄Hugging Face
login(token=os.environ["HUGGINGFACE_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


這邊我使用huggingface 的大模型，有想用其他商業大模型的可以參考原[教學](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/)

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline

model_id = "MediaTek-Research/Breeze-7B-Instruct-v1_0"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False, token=my_read_token)


model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto",torch_dtype=torch.bfloat16)

model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/508M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(61952, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

原教學先直接使用模型。ChatModels是LangChain“Runnables”的實例，這意味著它們公開了一個標準介面來與之互動。為了簡單地呼叫模型，langchain可以將訊息列表傳遞給該.invoke方法。

**補充說明:其實 messages這個只是把訊息貼在一起，可以看成簡單的A+B這樣，HumanMessage 跟SystemMessage的區別只是添加的貼合格式不同。**

**.invoke的方法就是huggface 產生文本的方法，應該只有格式略有不同，商業的模型我想應該有根據langchain的格式做微調或調整，開源的使用就很容易有點狀況。**



In [25]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_huggingface import HuggingFacePipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100, top_k=50)
llm = HuggingFacePipeline(pipeline=pipe)
messages = [
    HumanMessage(content="Hello!"),
    SystemMessage(content="幫我將前方文本翻譯成法文"),
]



llm.invoke(messages)


'Human: Hello!\nSystem: 幫我將前方文本翻譯成法文。\nHuman: Hello!\nTranslation: Bonjour!'

API 參考：[HumanMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html) |[系統訊息](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html)

請注意，模型的響應是一個AIMessage。其中包含字串回應以及有關回應的其他元資料。通常我們可能只想處理字串回應。我們可以使用一個簡單的輸出解析器來解析這個回應。

我們首先導入簡單的輸出解析器。

In [19]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

API 參考：[StrOutputParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)

你可以單獨使用它。例如，我們可以保存語言模型呼叫的結果，然後將其傳遞給解析器。



In [26]:
result = llm.invoke(messages)

parser.invoke(result)

'Human: Hello!\nSystem: 幫我將前方文本翻譯成法文。\nHuman: Hello!\nTranslation: Bonjour!'

更方邊的方法是可以使用此輸出解析器“連結”模型。這意味著該輸出解析器將在該鏈中每次被調用。此鏈採用語言模型的輸入類型（字串或訊息清單）並傳回輸出解析器的輸出類型（字串）。

我們可以使用操作符輕鬆建立鏈|。|LangChain中使用運算符將兩個元素組合在一起。

In [14]:
chain = llm | parser
chain.invoke(messages)

'Human: Hello!\nSystem: 幫我將前方文本翻譯成法文。\nHuman: Hello!\nTranslation: Bonjour!'

如果我們現在看一下 LangSmith，我們可以看到該chain有兩個步驟：首先呼叫語言模型，然後將結果傳遞到輸出解析器。

##Prompt 模板

現在我們將訊息列表直接傳遞到語言模型中。這個訊息列表從哪裡來？通常，它是由使用者輸入和應用程式邏輯的組合建構的。此應用程式邏輯通常會取得原始使用者輸入並將其轉換為準備傳遞到語言模型的訊息清單。常見的轉換包括新增系統訊息或使用使用者輸入格式化範本。

PromptTemplates是LangChain中的一個概念，旨在協助這種轉換。它們接收原始使用者輸入並傳回準備傳遞到語言模型的資料（提示）。

讓我們在這裡創建一個 PromptTemplate。它將接受兩個用戶變數：


- language：將文字翻譯成的語言

- text：要翻譯的文本

API 參考：[ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)


首先，我們建立一個字串，將其格式化為系統訊息：

接下來，我們可以建立 PromptTemplate。這將是一個組合system_template以及一個更簡單的模板，用於放置要翻譯的文本

In [15]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "將前方的中文翻譯成{language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [ ("user", "{text}"),("system", system_template)]
)
result = prompt_template.invoke({"language": "法文", "text": "你好!"})

result

ChatPromptValue(messages=[HumanMessage(content='你好!'), SystemMessage(content='將前方的中文翻譯成法文:')])

In [16]:
result.to_messages()

[HumanMessage(content='你好!'), SystemMessage(content='將前方的中文翻譯成法文:')]

In [17]:
chain = prompt_template | llm | parser

In [18]:
chain.invoke({"language": "法文", "text": "你好!"})

'Human: 你好!\nSystem: 將前方的中文翻譯成法文: 你好!\n中文: 你好!\n法文: Bonjour!'

後面[官方教學](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/)還有使用langServe 提供服務，不過我就不翻譯了。